### 데이터에서 지도 시각화 및 json 다루기
- price(단위: 만원): 집 값 평균
- population(단위: 명): 인구
- area(단위: $\text{km}^2$): 면적

> 참고 자료 및 데이터 : <br>
> geo data - https://github.com/southkorea/southkorea-maps <br>
> hangjeongdong - https://github.com/raqoon886/Local_HangJeongDong <br>
> 공공 데이터 - https://github.com/gisman/public-data <br>


In [415]:
import numpy as np
import pandas as pd
import folium       
import json

In [416]:
columns_busan = "price","population","area"
data_busan = {
    "gu" : ["중구", "서구", "동구", "영도구", "부산진구", "동래구", "남구", "북구", "해운대구", "사하구", "금정구", "강서구", "연제구", "수영구", "사상구", "기장군"],
    "price" : [15515, 28665, 20729, 17024, 28781, 35607, 29597, 23090, 34649, 17942, 29778, 30080, 30590, 39199, 19993, 22341],
    "population" : [44852, 112621, 89144, 121934, 365337, 267735, 286093, 299547, 414611, 332765, 247725, 128611, 208844, 178028, 229010, 167162],
    "area" : [2.83, 13.98, 9.74, 14.20, 29.67, 16.63, 26.81, 39.37, 51.47, 41.77, 65.28, 181.49, 12.10, 10.21, 36.09, 218.30]
}

df = pd.DataFrame(data=data_busan).set_index("gu")
df

,price,population,area
gu,,,
중구,15515,44852,2.83
서구,28665,112621,13.98
동구,20729,89144,9.74
영도구,17024,121934,14.20
부산진구,28781,365337,29.67
동래구,35607,267735,16.63
남구,29597,286093,26.81
북구,23090,299547,39.37
해운대구,34649,414611,51.47


In [417]:
geo_path = "busan_gu.json"
geo_str = json.load(open(geo_path, encoding='utf-8'))

In [418]:
print(f"""\
키 목록 : {geo_str.keys()}
피쳐 키목록 :{geo_str["features"][0].keys()}
id 확인 : {geo_str["features"][0]["id"]}\
""")

키 목록 : dict_keys(['type', 'features'])
피쳐 키목록 :dict_keys(['type', 'id', 'properties', 'geometry'])
id 확인 : 중구


In [419]:
# 구 이름 출력
for info in geo_str["features"]:
    print(info["id"])

중구
서구
동구
영도구
부산진구
동래구
남구
북구
해운대구
사하구
금정구
강서구
연제구
수영구
사상구
기장군


### 지도 시각화

In [420]:
map = folium.Map(location = [35.1856505, 129.1], zoom_start=10)

In [421]:
folium.Choropleth(geo_data = geo_str, 
               data = df["population"],
               fill_color = "PuRd",
               key_on = "feature.id",
               legend_name = "부산 인구 수(단위: 명)").add_to(map)

map

In [422]:
# 기준 좌표 설정
base_lat, base_lon = 35.1856505, 129.1

# 랜덤 데이터 생성 
range_= 0.1
marker_pick = [(base_lat + np.random.uniform(-range_, range_), base_lon + np.random.uniform(-range_, range_)) for _ in range(16)]

In [423]:
df["marker_pick"] = marker_pick
df

,price,population,area,marker_pick
gu,,,,
중구,15515,44852,2.83,"(35.24696652733657, 129.13397313462394)"
서구,28665,112621,13.98,"(35.137031157322205, 129.13153553398917)"
동구,20729,89144,9.74,"(35.16226798550428, 129.04855336648103)"
영도구,17024,121934,14.20,"(35.20023927516581, 129.18947337071583)"
부산진구,28781,365337,29.67,"(35.20386878479257, 129.0671514088949)"
동래구,35607,267735,16.63,"(35.15125419593577, 129.05039277263006)"
남구,29597,286093,26.81,"(35.26145659277401, 129.07296060462053)"
북구,23090,299547,39.37,"(35.211886418496526, 129.00097425334462)"
해운대구,34649,414611,51.47,"(35.24982983702988, 129.14241231702846)"


In [424]:
## iterrows()를 이용하여 31개 경찰서의 Marker를 표시하고, arrest_total을 반영한 Circle을 그리시오

for n,coor in zip(df.index, df["marker_pick"]):
    print(n,coor)
    folium.Marker(location = coor,
                  icon=folium.Icon(color='red', icon='home'),
                  popup=n).add_to(map)
    folium.Circle(location = coor,
                  radius = 10,
                  fill=True).add_to(map)
map

중구 (35.24696652733657, 129.13397313462394)
서구 (35.137031157322205, 129.13153553398917)
동구 (35.16226798550428, 129.04855336648103)
영도구 (35.20023927516581, 129.18947337071583)
부산진구 (35.20386878479257, 129.0671514088949)
동래구 (35.15125419593577, 129.05039277263006)
남구 (35.26145659277401, 129.07296060462053)
북구 (35.211886418496526, 129.00097425334462)
해운대구 (35.24982983702988, 129.14241231702846)
사하구 (35.19654193890152, 129.1995248829338)
금정구 (35.126714778798494, 129.0567175654805)
강서구 (35.25899516685692, 129.16624598390572)
연제구 (35.2163062376041, 129.1571888593274)
수영구 (35.26866081529033, 129.10891500954676)
사상구 (35.17303564608618, 129.0710937461264)
기장군 (35.08604548391357, 129.03276444404295)
